## Image Creation with Deep Convolutional GANs(Generative Adversarial Networks)

#### Part 1: Data Preprocessing

In [3]:
# Importing the libraries
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

In [4]:
# Setting some hyperparameters
batchSize = 64 # set the size of the batch.
imageSize = 64 # set the size of the generated images (64x64).

In [5]:
# Creating the transformations
transform = transforms.Compose([transforms.Scale(imageSize), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),]) # create a list of transformations (scaling, tensor conversion, normalization) to apply to the input images.

C:\Users\amira\Anaconda3\envs\face\lib\site-packages\torchvision\transforms\transforms.py:188: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


In [6]:
# Loading the dataset
dataset = dset.CIFAR10(root = './data', download = True, transform = transform) # download the training set in the ./data folder and i apply the previous transformations on each image.
dataloader = torch.utils.data.DataLoader(dataset, batch_size = batchSize, shuffle = True, num_workers = 2) # I use dataLoader to get the images of the training set batch by batch.

Files already downloaded and verified


#### Part 2: Building a Model

In [7]:
# Defining the weights_init function that takes as input a neural network m and that will initialize all its weights.
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

In [8]:
# Defining the generator

class G(nn.Module): #  introduce a class to define the generator.

    def __init__(self): #  introduce the __init__() function that will define the architecture of the generator.
        super(G, self).__init__() #  inherit from the nn.Module tools.
        self.main = nn.Sequential( #  create a meta module of a neural network that will contain a sequence of modules (convolutions, full connections, etc.).
            nn.ConvTranspose2d(100, 512, 4, 1, 0, bias = False), #  start with an inversed convolution.
            nn.BatchNorm2d(512), #  normalize all the features along the dimension of the batch.
            nn.ReLU(True), #  apply a ReLU rectification to break the linearity.
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias = False), # add another inversed convolution.
            nn.BatchNorm2d(256), # normalize again.
            nn.ReLU(True), #  apply another ReLU.
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias = False), #  add another inversed convolution.
            nn.BatchNorm2d(128), #  normalize again.
            nn.ReLU(True), # apply another ReLU.
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias = False), #  add another inversed convolution.
            nn.BatchNorm2d(64), #  normalize again.
            nn.ReLU(True), # We apply another ReLU.
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias = False), #  add another inversed convolution.
            nn.Tanh() # apply a Tanh rectification to break the linearity and stay between -1 and +1.
        )
        
    def forward(self, input): #  define the forward function that takes as argument an input that will be fed to the neural network, and that will return the output containing the generated images.
        output = self.main(input) #  forward propagate the signal through the whole neural network of the generator defined by self.main.
        return output #  return the output containing the generated images.

In [9]:
# Creating the generator
netG = G() #  create the generator object.
netG.apply(weights_init) # initialize all the weights of its neural network.

G(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
    (2): ReLU(inplace)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (5): ReLU(inplace)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (8): ReLU(inplace)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (11): ReLU(inplace)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

In [10]:
# Defining the discriminator

class D(nn.Module): #  introduce a class to define the discriminator.

    def __init__(self): #  introduce the __init__() function that will define the architecture of the discriminator.
        super(D, self).__init__() #  inherit from the nn.Module tools.
        self.main = nn.Sequential( #  create a meta module of a neural network that will contain a sequence of modules (convolutions, full connections, etc.).
            nn.Conv2d(3, 64, 4, 2, 1, bias = False), #  start with a convolution.
            nn.LeakyReLU(0.2, inplace = True), #  apply a LeakyReLU.
            nn.Conv2d(64, 128, 4, 2, 1, bias = False), # We add another convolution.
            nn.BatchNorm2d(128), #  normalize all the features along the dimension of the batch.
            nn.LeakyReLU(0.2, inplace = True), #  apply another LeakyReLU.
            nn.Conv2d(128, 256, 4, 2, 1, bias = False), #  add another convolution.
            nn.BatchNorm2d(256), #  normalize again.
            nn.LeakyReLU(0.2, inplace = True), #  apply another LeakyReLU.
            nn.Conv2d(256, 512, 4, 2, 1, bias = False), #  add another convolution.
            nn.BatchNorm2d(512), #  normalize again.
            nn.LeakyReLU(0.2, inplace = True), #  apply another LeakyReLU.
            nn.Conv2d(512, 1, 4, 1, 0, bias = False), # We add another convolution.
            nn.Sigmoid() #  apply a Sigmoid rectification to break the linearity and stay between 0 and 1.
        )

    def forward(self, input): #  define the forward function that takes as argument an input that will be fed to the neural network, and that will return the output which will be a value between 0 and 1.
        output = self.main(input) #  forward propagate the signal through the whole neural network of the discriminator defined by self.main.
        return output.view(-1) # return the output which will be a value between 0 and 1.

In [11]:
# Creating the discriminator
netD = D() # create the discriminator object.
netD.apply(weights_init) # initialize all the weights of its neural network.

D(
  (main): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): LeakyReLU(0.2, inplace)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (3): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
    (4): LeakyReLU(0.2, inplace)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (6): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
    (7): LeakyReLU(0.2, inplace)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (9): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
    (10): LeakyReLU(0.2, inplace)
    (11): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (12): Sigmoid()
  )
)

#### Part 3: Training and Testing the Model

In [ ]:
# Training the DCGANs

criterion = nn.BCELoss() #  create a criterion object that will measure the error between the prediction and the target.
optimizerD = optim.Adam(netD.parameters(), lr = 0.0002, betas = (0.5, 0.999)) #  create the optimizer object of the discriminator.
optimizerG = optim.Adam(netG.parameters(), lr = 0.0002, betas = (0.5, 0.999)) #  create the optimizer object of the generator.

for epoch in range(25): #  iterate over 25 epochs.

    for i, data in enumerate(dataloader, 0): #  iterate over the images of the dataset.
        
        # 1st Step: Updating the weights of the neural network of the discriminator

        netD.zero_grad() #  initialize to 0 the gradients of the discriminator with respect to the weights.
        
        # Training the discriminator with a real image of the dataset
        real, _ = data #  get a real image of the dataset which will be used to train the discriminator.
        input = Variable(real) #  wrap it in a variable.
        target = Variable(torch.ones(input.size()[0])) #  get the target.
        output = netD(input) # forward propagate this real image into the neural network of the discriminator to get the prediction (a value between 0 and 1).
        errD_real = criterion(output, target) #  compute the loss between the predictions (output) and the target (equal to 1).
        
        # Training the discriminator with a fake image generated by the generator
        noise = Variable(torch.randn(input.size()[0], 100, 1, 1)) # make a random input vector (noise) of the generator.
        fake = netG(noise) #  forward propagate this random input vector into the neural network of the generator to get some fake generated images.
        target = Variable(torch.zeros(input.size()[0])) # get the target.
        output = netD(fake.detach()) # forward propagate the fake generated images into the neural network of the discriminator to get the prediction (a value between 0 and 1).
        errD_fake = criterion(output, target) # compute the loss between the prediction (output) and the target (equal to 0).

        # Backpropagating the total error
        errD = errD_real + errD_fake #compute the total error of the discriminator.
        errD.backward() # backpropagate the loss error by computing the gradients of the total error with respect to the weights of the discriminator.
        optimizerD.step() # apply the optimizer to update the weights according to how much they are responsible for the loss error of the discriminator.
        
         # 2nd Step: Updating the weights of the neural network of the generator

        netG.zero_grad() #  initialize to 0 the gradients of the generator with respect to the weights.
        target = Variable(torch.ones(input.size()[0])) #  get the target.
        output = netD(fake) #  forward propagate the fake generated images into the neural network of the discriminator to get the prediction (a value between 0 and 1).
        errG = criterion(output, target) #  compute the loss between the prediction (output between 0 and 1) and the target (equal to 1).
        errG.backward() #  backpropagate the loss error by computing the gradients of the total error with respect to the weights of the generator.
        optimizerG.step() #  apply the optimizer to update the weights according to how much they are responsible for the loss error of the generator.